In [1]:
# compare default ODIAC and EDGAR5, and apply seasonal cycle

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

from matplotlib import rcParams
rcParams["savefig.dpi"] = 300
rcParams["font.size"] = 12

import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from util.functions import area_latlon
from util.gamap_colormap import WhGrYlRd

In [4]:
input_dir = '/home/msadiq/Desktop/co2/emissions/'

In [5]:
ds_GC = xr.open_dataset(input_dir + 'GC/nc/ODIAC_CO2.monthly.generic.1x1.nc')
ds_EDGAR = xr.open_dataset(input_dir + 'EDGAR/nc/v50_CO2_excl_short-cycle_org_C_2014.0.1x0.1.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/home/msadiq/Desktop/co2/emissions/GC/nc/ODIAC_CO2.monthly.generic.1x1.nc'

In [ ]:
ds_GC['CO2'][168:180,:,:]

In [ ]:
ds_EDGAR['emi_co2']

In [ ]:
# 1, calculate annual total emission (Tg/yr) and compare
# need function to compute grid area

# 1.1, EDGAR total
data = ds_EDGAR['emi_co2'].copy()
lat_res = 0.1
lon_res = 0.1

for ilat, lat in enumerate(data.lat):
    area_tmp = area_latlon(lat1 = lat, 
                           lon1 = data.lon[10].values, 
                           lat2 = lat + lat_res, 
                           lon2 = data.lon[10].values + lon_res)
    data[ilat,:] = data[ilat,:] * area_tmp *3600*24*365*1e6 # unit: kgCO2/yr
annual_edgar = data.copy()
total_edgar = data.sum() # kgCO2/year, same as data description
print('Annual total ffCO2 emission (GtCO2/yr): ', total_edgar.values*1e-12) # GtCO2/yr
print('Annual total ffCO2 emission (GtC/yr): ', total_edgar.values*1e-12*12/44) # GtCO2/yr

In [ ]:
# 1.2, GC total
data = ds_GC['CO2'][168:180,:,:].copy()
lat_res = 1
lon_res = 1
days_in_month = [31,28,31,30,31,30,31,31,30,31,30,31]

for ilat, lat in enumerate(data.lat):
    for imonth in range(12):
        area_tmp = area_latlon(lat1 = lat, 
                           lon1 = data.lon[10].values, 
                           lat2 = lat + lat_res, 
                           lon2 = data.lon[10].values + lon_res)
        data[imonth,ilat,:] = data[imonth,ilat,:] * area_tmp *days_in_month[imonth] *3600*24*1e6 # unit: kgCO2/yr
annual_gc = data.sum(dim='time')
total_gc = data.sum() # kgCO2/year, same as data description
total_gc*1e-12 # GtCO2/yr

In [ ]:
print('difference: ', 100*(total_edgar-total_gc)/total_gc, '%')

In [ ]:
# 2, regrid EDGAR to 1x1 degree res and compare with GC
ds_EDGAR_1degree = xr.open_dataset(input_dir + 
                                   'EDGAR/nc/EDGAR_2018_1x1.nc')
# 2.1 EDGAR annual total from 1 degree
data = ds_EDGAR_1degree['emi_co2'].copy() # unit: kg/m2/s
lat_res = 1
lon_res = 1

for ilat, lat in enumerate(data.lat):
    area_tmp = area_latlon(lat1 = lat, 
                           lon1 = data.lon[10].values, 
                           lat2 = lat + lat_res, 
                           lon2 = data.lon[10].values + lon_res)
    data[ilat,:] = data[ilat,:] * area_tmp *3600*24*365*1e6 # unit: kgCO2/yr
annual_edgar = data.copy()
total_edgar = data.sum() # kgCO2/year, same as data description
total_edgar*1e-12 # GtCO2/yr

In [ ]:
diff = (annual_edgar - annual_gc)/annual_gc
#diff = annual_edgar/annual_gc
#diff = diff*1e-9 # MtCO2/yr

fig = plt.figure(figsize=[8, 4])
proj=ccrs.PlateCarree()
ax = plt.subplot(111, projection=proj)
ax.coastlines()
diff.plot(ax=ax, transform=ccrs.PlateCarree(), cmap = WhGrYlRd, vmax = 5, cbar_kwargs={'shrink': 0.8, 'label': ''})
ax.set_title('EDGAR - ODIAC');

In [ ]:
# 3, pick a grid cell and compare seasonal cycle between the two
loc_lat = 48
loc_lon = 2

one_edgar = ds_EDGAR_1degree['emi_co2'].sel(lat=loc_lat,lon=loc_lon, method='nearest')
one_gc    = ds_GC['CO2'][168:180,:,:].sel(lat=loc_lat,lon=loc_lon, method='nearest')

fig = plt.figure(figsize=[7, 4])
plt.title('Monthly CO2 emission rate')
plt.plot(range(1,13), one_gc, 'b--', label = 'GEOS-Chem')
plt.plot(range(1,13), np.repeat(one_gc.mean().values,12), 'b', label = 'GEOS-Chem mean')
plt.plot(range(1,13), np.repeat(one_edgar.values,12), 'r', label = 'EDGAR')
plt.ylabel('emission rate (kg m-2 s-1)')
plt.xlabel('month')
plt.legend()

fig.savefig(input_dir + '/comparison/EDGAR_monthly.png', dpi=300)

In [ ]:
scale_f = one_gc / one_gc.mean()
scaled_edgar = scale_f * one_edgar
plt.plot(scaled_edgar)
plt.plot(one_gc)

In [ ]:
# 4, add seasonal scaling factor from GC to EDGAR dataset
empty_edgar_12 = ds_GC['CO2'][168:180,:,:].copy()
empty_edgar_12.values = np.zeros(12*180*360).reshape(12,180,360)
empty_edgar_12.attrs['long_name'] = 'EDGAR fossil fuel CO2 from Mehliyar Sadiq'

In [ ]:
for ilon in range(360):
    for ilat in range(180):
        one_edgar = ds_EDGAR_1degree['emi_co2'].isel(lat = ilat, lon = ilon)
        one_gc    = ds_GC['CO2'][168:180,:,:].isel(lat=ilat,lon=ilon)
        scale_f = one_gc / one_gc.mean()
        empty_edgar_12[:,ilat,ilon] = scale_f * one_edgar

In [ ]:
tmp = empty_edgar_12.sel(lat=loc_lat,lon=loc_lon, method='nearest')
plt.plot(tmp)

In [ ]:
edgar_monthly = empty_edgar_12.to_dataset(name='emi_co2')
print(edgar_monthly)
edgar_monthly.to_netcdf(input_dir+'EDGAR/nc/EDGAR_monthly_2018_1x1.nc')